In [48]:
import re
import numpy as np
import nltk
import pandas as pd
import tqdm
import spacy
nltk.download('stopwords') 
nltk.download('wordnet') 

[nltk_data] Downloading package stopwords to C:\Users\Leo
[nltk_data]     Hirano\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Leo
[nltk_data]     Hirano\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [49]:
data_tip=pd.read_json('./data/yelp_academic_dataset_tip.json', lines=True,)

In [122]:
# 2000 reviews set

data_review = pd.read_json('./data/yelp_academic_dataset_review.json', lines=True, nrows=2000)

In [335]:
# full reviews set

chunk_size = 100000

chunks = []

for chunk in pd.read_json('./data/yelp_academic_dataset_review.json', lines=True, chunksize=chunk_size):
    chunks.append(chunk)

data_review_total = pd.concat(chunks, ignore_index=True)


In [180]:
data_review_total.describe()


,stars,useful,funny,cool,date
count,6.990280e+06,6.990280e+06,6.990280e+06,6.990280e+06,6990280
mean,3.748584e+00,1.184609e+00,3.265596e-01,4.986175e-01,2017-01-11 11:22:33.441780992
min,1.000000e+00,-1.000000e+00,-1.000000e+00,-1.000000e+00,2005-02-16 03:23:22
25%,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2015-01-25 04:53:50.249999872
50%,4.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2017-06-03 01:26:07
75%,5.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,2019-05-23 00:02:46.249999872
max,5.000000e+00,1.182000e+03,7.920000e+02,4.040000e+02,2022-01-19 19:48:45
std,1.478705e+00,3.253767e+00,1.688729e+00,2.172460e+00,NaN


In [181]:
data_users = pd.read_json('./data/yelp_academic_dataset_user.json', lines=True)


In [191]:
# reviews of only the top 200 reviewers

sorted_users = data_users.sort_values(by='review_count', ascending=False)
top_reviewers = sorted_users.head(200)
top_reviewers
data_top_reviewers = data_review_total[data_review_total['user_id'].isin(top_reviewers['user_id'])]

data_top_reviewers.describe()


,stars,useful,funny,cool,date
count,33027.000000,33027.000000,33027.000000,33027.000000,33027
mean,3.832985,6.642020,2.935961,4.993157,2015-08-23 18:46:18.560722944
min,1.000000,0.000000,0.000000,0.000000,2005-04-29 04:34:01
25%,3.000000,2.000000,0.000000,1.000000,2012-12-10 12:38:05
50%,4.000000,4.000000,1.000000,3.000000,2015-12-08 02:13:25
75%,4.000000,8.000000,3.000000,6.000000,2018-07-31 06:57:50.500000
max,5.000000,154.000000,139.000000,149.000000,2022-01-19 17:48:38
std,0.914711,8.787875,5.262922,7.795310,NaN


In [51]:
data_business=pd.read_json('./data/yelp_academic_dataset_business.json', lines=True,)

In [54]:
from gensim.parsing.preprocessing import STOPWORDS

In [56]:
data_review.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [57]:
data_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [58]:
# text processing

def process_text(document,stopwords=STOPWORDS):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy=nlp(document)
    lemma_txt = ["".join(token.lemma_) for token in tqdm.tqdm(doc_spacy)]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in STOPWORDS]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return lemma_no_stop_txt

In [59]:
review_text = data_review['text']

In [192]:
import spacy
import re
import tqdm

def process_text_no_punctuation(document, stopwords=[]):
    nlp = spacy.load("en_core_web_sm")
    document = re.sub(r'\s+', ' ', document, flags=re.I) 
    document = re.sub(r'\W', ' ', str(document)) 
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document) 
    document = document.lower() 
    doc_spacy = nlp(document)
    lemma_txt = [token.lemma_ for token in tqdm.tqdm(doc_spacy)]
    lemma_no_stop_txt = [word for word in lemma_txt if word not in stopwords]
    lemma_no_stop_txt = [word for word in lemma_no_stop_txt if len(word) > 3]
    clean_txt = ' '.join(lemma_no_stop_txt)
    return clean_txt

In [193]:
top_reviewers_text_preprocessed = data_top_reviewers['text'].apply(process_text)

100%|██████████| 270/270 [00:00<?, ?it/s]


In [ ]:
review_text = data_review['text']

In [63]:
review_text_preprocessed = review_text.apply(process_text)

100%|██████████| 43/43 [00:00<?, ?it/s]


In [95]:
file_path = "unprocessed_2000.csv"

# Export DataFrame to CSV
review_text_preprocessed.to_csv(file_path, index=False)

In [334]:
file_path = "top_reviews_processed.csv"

# Export DataFrame to CSV
top_pre.to_csv(file_path, index=False)

In [337]:
user_counts = data_top_reviewers['user_id'].value_counts()

# Find the user IDs with the most entries
max_entries_users = user_counts[user_counts == user_counts.max()].index.tolist()

print("User IDs with the most entries:")
for user_id in max_entries_users:
    print(user_id)

User IDs with the most entries:
_BcWyKQL16ndpBdggh2kNA


In [240]:
# returns all preproccessed reviews for a user id

def user_preprocessed(user_id):
    reviews_df = data_top_reviewers

    preprocessed_texts = top_reviewers_text_preprocessed

    user_reviews_df = reviews_df[reviews_df['user_id'] == user_id].copy()

    user_reviews_df['preprocessed_text'] = preprocessed_texts[user_reviews_df.index]
    return user_reviews_df[['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'preprocessed_text', 'date']]

#print(user_preprocessed('fr1Hz2acAb3OaL3l6DyKNg')['preprocessed_text'].head())


In [232]:

top_pre = pd.DataFrame(top_reviewers_text_preprocessed)
top_pre.head()


123    [decide, need, door, green, project, bunch, le...
197    [airport, grab, restaurant, earl, decent, vari...
351    [candy, california, base, bakery, reach, flori...
396    [estimate, plumber, high, quote, plumber, inte...
457    [group, office, outing, lunch, number, people,...
Name: text, dtype: object

In [340]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import string

# Function to remove punctuation from a string
def remove_punctuation(text):
    return ''.join([char for char in text if char not in string.punctuation])

# Update the preprocessing step to remove punctuation
all_reviews = [''.join([remove_punctuation(token) for token in review]) for review in top_pre['text']]

def similar_reviews(user_id):
    tfidf_review = tfidf_vectorizer.fit_transform(all_reviews)
    
    user_reviews = user_preprocessed(user_id)['preprocessed_text']
    user_reviews_strings = [''.join([remove_punctuation(token) for token in review]) for review in user_reviews]
    tfidf_user_reviews = tfidf_vectorizer.transform(user_reviews_strings)

    user_review_cos = cosine_similarity(tfidf_user_reviews, tfidf_review)

    sorted_review_indices = sorted(range(len(user_review_cos[0])), key=lambda i: user_review_cos[0][i], reverse=True)[:10]

    return sorted_review_indices


print("similar:")
data_top_reviewers.iloc[similar_reviews('fr1Hz2acAb3OaL3l6DyKNg')]




similar:


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
351,s9ieTQEl3ImVSkTRrP4-3g,fr1Hz2acAb3OaL3l6DyKNg,T3muepcwD45gHLWcpaNNDQ,4,5,1,7,"See's candies is California base bakery reach Florida. I know about this candies when my wife living in California. Their chocolate is very soft and tasty. \n\nIn Tampa, the International Plaza Ma...",2016-12-04 05:36:48
123,suMtrxm_nUiAQMu6ZfG5Wg,0Igx-a1wAstiBDerGxXk2A,actDmpJyaQPnkds6knMdKA,5,3,2,1,"Say you decide that you need a new door...and you only have $35. Where do you go? the Green Project. \n\nOr you have a bunch of left over building supplies, random bits of lumber, some roofing m...",2009-03-01 05:30:45
197,IJCYT9dRjuBvY-BJufFnNg,hWDybu_KvYLSdEFzGrniTw,roc-xyyzN5b_nImb0NdRLQ,4,6,5,5,"As airport grab-and-go restaurants are, Earl does a decent job with a variety of sandwich options including turkey, tuna and even a Hawaiian. But today I could smell the roast beef and so I got th...",2015-01-03 23:16:09
396,o_NE6d5LmUEfP-TMZx5IZg,_BcWyKQL16ndpBdggh2kNA,tD_2Pw2kiUGE-lIkFTQKLw,3,2,1,0,Got an estimate but went with another plumber as this was the highest quote of the 3 plumbers we interviewed for the job.\nVery prompt communication though.\n\nWill re-consider for future jobs. S...,2015-02-02 17:43:00
457,nhfB-ssDwJxbpAUjlE889A,fr1Hz2acAb3OaL3l6DyKNg,t-bcEvAM8oE92tm9HBr7wQ,3,8,3,9,"We have big group of 12 for the office outing lunch. I would said for the number of people we have, the service is not the best but good enough. Only one of our members get the food late. The clos...",2017-05-26 03:20:52
605,7Ki1tkHP23P2i--9qsTRdg,-G7Zkl1wIWBBmD0KRy_sCw,IbndcMURguByburM72o3SA,5,9,6,4,"Just finished a State Street Kitchen, Grilled Chicken, Spinach, Swiss and Honey Mustard Sub. To kick it up a notch I stuffed in Jalapeno Chips for a spicy crunch. \nIt was great on an afternoon w...",2013-10-24 18:13:21
626,p87acJKiGDY07_njsTefLg,qjfMBIZpQT9DDtw_BWCopQ,KhBUg5QhBYuK8RZAe5gDMQ,4,4,2,1,"I love the concept of this place. One half of it a café that serves different types of coffees and teas, and breakfast type items, and the other world where it serves bar type apps, salads, and sa...",2015-02-02 20:15:22
658,4NlHWuh192f4Z-6FIxgxGQ,Um5bfs5DH6eizgjH3xZsvg,8tnwneWHXWLUvIc5zz__5A,3,2,1,1,I stopped at Walgreens in hopes of seeing the new ELF Villain's collection. Sometimes Walgreens gets them before the holidays. Sometimes they don't. This particular one doesn't ever get them accor...,2015-10-08 04:51:22
798,1CvkiydhF66ha7G6BG2moQ,B-s-8VUnuBjGTP3d01jsyw,m1EGAIfFEN9nG2_bJqnAcQ,4,3,1,4,"That horrible angst - you want sushi, but you're not SO hungry that you can do AYCE or want to spend $25 for lunch. So we popped in on a weekend for a late lunch and decided to order a la carte. ...",2015-04-01 18:19:11
876,XJwl30x7X22v2aKH4Q7GBQ,_BcWyKQL16ndpBdggh2kNA,bJAY2baMKSTlWRc-QZGopQ,3,2,2,1,"I like the Neuvo-Latino-Americano grub here. The place has moved locations and is now a bit larger. Chips & Salsa are always fresh 7 tasty, too.\n\nI wish I could give a more substanital review ...",2008-12-04 13:49:58


In [323]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()

def find_similar_users(user_id):
    tfidf_reviews = tfidf_vectorizer.fit_transform(all_reviews)

    user_reviews = user_preprocessed(user_id)['preprocessed_text']
    user_reviews_strings = [''.join([remove_punctuation(token) for token in review]) for review in user_reviews]
    tfidf_user_reviews = tfidf_vectorizer.transform(user_reviews_strings)

    user_review_cos = cosine_similarity(tfidf_user_reviews, tfidf_reviews)

    sorted_user_indices = sorted(range(len(user_review_cos[0])), key=lambda i: user_review_cos[0][i], reverse=True)

    most_similar_user_index = sorted_user_indices[3]

    most_similar_user_ids = data_top_reviewers.iloc[most_similar_user_index]['user_id']

    return most_similar_user_ids

similar_user_id = find_similar_users('hWDybu_KvYLSdEFzGrniTw')
print("Most similar user ID:", similar_user_id)

user_data = user_preprocessed('hWDybu_KvYLSdEFzGrniTw')['preprocessed_text']
similar_user_data = user_preprocessed(similar_user_id)['preprocessed_text']

pd.set_option('display.max_colwidth', 200)

print(user_data.head())
print(similar_user_data.head())

pd.set_option('display.max_colwidth', 200)



Most similar user ID: _BcWyKQL16ndpBdggh2kNA
197      [airport, grab, restaurant, earl, decent, variety, sandwich, option, include, turkey, tuna, hawaiian, today, smell, roast, beef, good, portion, sliced, beef, medium, cook, cheddar, cheese, yummy, ...
41695    [germantown, sport, land, watch, superbowl, limit, menu, choose, wing, deliver, polite, friendly, waitress, wing, actually, flavor, instead, taste, like, meaty, cook, properly, good, little, mushy...
57699    [waffle, house, today, experience, great, stack, couple, customer, service, slow, waffle, overdone, think, underdone, coffee, overdone, crew, loud, complain, understaffe, manager, special, board, ...
84381    [menu, item, cheese, burger, hoagie, burger, fine, hoagie, roll, steam, mushy, inside, firm, outside, like, happen, microwave, bread, suggest, thing, suggestion, stick, cheesesteak, consume, look,...
96386    [walk, baggage, claim, national, drive, away, stop, slam, star, review, minute, later, national, arrive, pick,

In [331]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

for attempt in range(50):
    random_user_id = np.random.choice(data_top_reviewers['user_id'])

    business_ids_user1 = set(data_top_reviewers[data_top_reviewers['user_id'] == random_user_id]['business_id'])

    similar_user_id = find_similar_users(random_user_id)  

    business_ids_user2 = set(data_top_reviewers[data_top_reviewers['user_id'] == similar_user_id]['business_id'])

    common_business_ids = business_ids_user1.intersection(business_ids_user2)

    user1_ratings = []
    user2_ratings = []

    for business_id in common_business_ids:
        user1_review = data_top_reviewers[(data_top_reviewers['user_id'] == random_user_id) & 
                                        (data_top_reviewers['business_id'] == business_id)]
        user2_review = data_top_reviewers[(data_top_reviewers['user_id'] == similar_user_id) & 
                                        (data_top_reviewers['business_id'] == business_id)]
        
        if not user1_review.empty and not user2_review.empty:
            user1_ratings.append(user1_review['stars'].values[0])
            user2_ratings.append(user2_review['stars'].values[0])

    if len(user1_ratings) > 0 and len(user2_ratings) > 0:
        user1_ratings_array = np.array(user1_ratings).reshape(1, -1)
        user2_ratings_array = np.array(user2_ratings).reshape(1, -1)
        
        similarity_score = cosine_similarity(user1_ratings_array, user2_ratings_array)[0, 0]
        
        print("Similarity Score (Cosine similarity):", similarity_score)
    else:
        print("No common ratings found. Skipping...")



Similarity Score (Cosine similarity): 0.9871352947927473
Similarity Score (Cosine similarity): 0.9856590736780187
No common ratings found. Skipping...
No common ratings found. Skipping...
Similarity Score (Cosine similarity): 0.9823362070472675
No common ratings found. Skipping...
Similarity Score (Cosine similarity): 0.9517173271012339
Similarity Score (Cosine similarity): 0.9999999999999998
No common ratings found. Skipping...
Similarity Score (Cosine similarity): 1.0
Similarity Score (Cosine similarity): 1.0
Similarity Score (Cosine similarity): 0.9391485505499116
Similarity Score (Cosine similarity): 1.0
No common ratings found. Skipping...
Similarity Score (Cosine similarity): 0.9938837346736189
No common ratings found. Skipping...
Similarity Score (Cosine similarity): 1.0
Similarity Score (Cosine similarity): 0.9517173271012339
Similarity Score (Cosine similarity): 0.9691168885159733
No common ratings found. Skipping...
Similarity Score (Cosine similarity): 0.9938837346736189
Sim

In [354]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


# Create TF-IDF matrix
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(pd.DataFrame(top_reviewers_text_preprocessed))



# Function to recommend businesses - unable to find results
def recommend_business(user_id):
    user_reviews = data_top_reviewers[data_top_reviewers['user_id'] == user_id]
    business_ids = data_top_reviewers['business_id'].unique()

    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

    sim_scores = []
    for business_id in business_ids:
        index = data_top_reviewers[data_top_reviewers['business_id'] == business_id].index
        print(index)
        if not index.empty:
            index = index[0]
        else:
            sim_scores.extend(list(cosine_sim[index]))

    sim_scores = [(i, score) for i, score in enumerate(sim_scores) if data_top_reviewers.iloc[i]['business_id'] not in business_ids]
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    top_similar_indices = [i[0] for i in sim_scores[:5]]
    top_similar_businesses = data_top_reviewers.iloc[top_similar_indices]['business_id'].unique()
    
    return top_similar_businesses

user_id = '_BcWyKQL16ndpBdggh2kNA'  
recommended_businesses = recommend_business(user_id)
print("Recommended businesses:", recommended_businesses)



Index([123, 204171], dtype='int64')
Index([197, 121751, 454189], dtype='int64')
Index([351], dtype='int64')
Index([396], dtype='int64')
Index([457], dtype='int64')
Index([605, 304834, 526702, 657397], dtype='int64')
Index([626, 61534, 314542, 488194, 638560], dtype='int64')
Index([658], dtype='int64')
Index([798], dtype='int64')
Index([876], dtype='int64')
Index([1097, 30976, 108636, 337760, 354292, 432457, 455614, 528456, 565516], dtype='int64')
Index([  1129,  43205,  67441,  70699, 103656, 106101, 123638, 139218, 253885,
       296919, 304819, 336369, 348527, 380411, 437908, 459815, 508941, 600357,
       606707],
      dtype='int64')
Index([1292, 114160, 140719, 190202, 503881, 603885, 627504, 679997], dtype='int64')
Index([1503, 56732, 124837, 238515, 625865], dtype='int64')
Index([1587, 52659], dtype='int64')
Index([1806], dtype='int64')
Index([1873], dtype='int64')
Index([1920, 175956], dtype='int64')
Index([3844], dtype='int64')
Index([3885], dtype='int64')
Index([3953, 293079,

In [348]:
import pandas as pd


business_ids_user1 = set(data_top_reviewers[data_top_reviewers['user_id'] == 'hWDybu_KvYLSdEFzGrniTw']['business_id'])

business_ids_user2 = set(data_top_reviewers[data_top_reviewers['user_id'] == similar_user_id]['business_id'])

common_business_ids = business_ids_user1.intersection(business_ids_user2)

print(common_business_ids)


{'I_3LMZ_1m2mzR0oLIOePIg', 'Smx7keqNQjEt23HgSPCbrw', 'SwBhaxfQPbyhsi0QHUAN0A', '6LAp-gSP9Kk4ZvxijXaxIw', 'RqW9S4WG9UYZHKhHRHXJZg', '0T5mai7FmUH06Tyv3rY17g', 'tIYd4pzWFlQasuX_JmYKBQ', 'qsLIuY5iERjJ_PJRwZVeRQ', 'Lw_mMFUYk4GmiyRF-iETbg', 'ACHVRhkmpCuT7WQQjzt07Q', '5eYrQfz--n8mas2_KfCp7Q', 'uoBmv4LIzOk6C-40MUVMuw', 'bqFG0QJY9jj2m55OqAVHeA', '-K0LoSCfh8i5U_y53Krepg', 'sAduA1AvV7Z27IGtsx_H3g', 'c_4c5rJECZSfNgFj7frwHQ', '_78Lam9Jd0W6ZmnHTvdjMg', '7L0WAmZ0fP5hKzdhVAkIhQ', 'jpi_v4T1LinzOVtng7OOwg', 'I0yR4DdzFF7AuyDgNfuAkQ', 'mY4i1N0Hm2fjIgU0Eyn11Q', 'Awkrb1wzFxz5w1Ixc9W3FA', 'bEn-Im7R7Ic4XDORq6_oSQ', '8O35ji_yOMVJmZ6bl96yhQ', 'Zre8y_XnmF0Pt2Lj5GhmVw'}


In [75]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

def recommend_restaurants(user_id, user_reviews, ratings):
    # Split user's reviews into training and testing data
    X_train, X_test, y_train, y_test = train_test_split(user_reviews, ratings, test_size=0.5, random_state=42)
    
    # Train Naive Bayes classifier with SVM
    classifier = Pipeline([
        ('tfidf', TfidfVectorizer()),
        ('clf', MultinomialNB()),
    ])
    classifier.fit(X_train, y_train)
    
    # Predict ratings for all restaurants
    all_restaurants_ratings = classifier.predict(all_restaurants_reviews)
    
    # Get indices of top 10 recommended restaurants
    top_indices = all_restaurants_ratings.argsort()[-10:][::-1]
    
    # Return list of top 10 recommended restaurants
    top_restaurants = [all_restaurants[i] for i in top_indices]
    return top_restaurants